In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [48]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (res):
                    R_backbone has       1750 params (~ 50.7) %
                    t_backbone has       1699 params (~ 49.3) %
total params:                 3449
cam2cam estimation => adding 3449 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@14.05.2021-20:28:53
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 7.789, TRANS ~ 68.503, POSE ~ 78918.125, ROTO ~ 9.218, 3D ~ 575.688, SELF R ~ 46.337, SELF t ~ 21.017, TOTAL ~ 832.200
#[cam2cam]: training batch iter 0 MPJPE: ~ 415.758 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 7.745, TRANS ~ 69.646, POSE ~ 84339.383, ROTO ~ 9.505, 3D ~ 598.941, SELF R ~ 53.853, SELF t ~ 14.375, TOTAL ~ 796.539
#[cam2cam]: training batch iter 1 MPJPE: ~ 472.054 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 8.361, TRANS ~ 67.916, POSE ~ 80179.031, ROTO ~ 8.729, 3D ~ 584.311, SELF R ~ 45.773, SELF t ~ 13.0